# **Task Lab [5]_AIE425_Intelligent Recommender Systems**
# Name: **Aser Mohamed Ali Mahmoud**
# ID: **222102487**
# Program: **AIS**

# The code adds an implementation to **Raw Cosine Similarity** and **Mean-Centered Cosine Similarity**

# Also, a calculation loops with methods to calculate the **Processing Time**, **Execution Time**, **TIME it took for each Loop**, and **Space Complexity**

# Ensuring that is no **Built-In Functions** or **Dot-Products**

# The Original Code Execution Summary: Clustering Algorithm Comparison

This code block executes a comprehensive comparison of three fundamental clustering algorithms: K-Means, K-Nearest Neighbors (KNN), and Decision Tree, specifically tailored for a Collaborative Filtering (CF) scenario using the MovieLens 100K dataset.

The primary focus is on **algorithm complexity and performance analysis**, with each custom-built model tracking its own execution time and operation count.

### Execution Flow:

1.  **Data Setup (Colab):** The code first handles the Colab environment by checking for and unzipping the `Movie_Lens_100K_Dataset.zip` file.
2.  **Feature Engineering:** It loads the MovieLens `u.data` file and creates a simplified 2-feature vector for each user: `[average rating, number of ratings]`.
3.  **Base Label Generation:** A standard `sklearn.KMeans` model is used to quickly generate a set of 5 "base clusters." These are used as the "ground truth" labels for training the supervised models (KNN and Decision Tree).
4.  **Training & Testing:** The data is split into training (80%) and testing (20%) sets.
5.  **Algorithm Comparison:**
    *   **K-Means Clustering (Custom):** Trained and tested.
    *   **K-Nearest Neighbors (Custom):** Trained and tested.
    *   **Decision Tree Clustering (Custom):** Trained and tested.
6.  **Performance Reporting:** For each algorithm, the output includes:
    *   **Actual Time:** Training and prediction time in seconds/milliseconds.
    *   **Operation Count:** A custom counter tracking the number of fundamental arithmetic operations.
    *   **Theoretical Complexity:** The expected Big O notation (e.g., O(n·k·d·t)) and a comparison to the actual operation count.



In [1]:
import os
import zipfile
import pandas as pd
import numpy as np
import time
from collections import Counter, defaultdict
import sys
from sklearn.cluster import KMeans as SKLearnKMeans
from sklearn.metrics.pairwise import cosine_similarity

# ═══════════════════════════════════════════════════════════════════════════════
# DATA LOADING
# ═══════════════════════════════════════════════════════════════════════════════
# Step 1: Upload your 'Movie_Lens_100K_Dataset.zip' file to the Colab session.
# It will appear in the path /content/Movie_Lens_100K_Dataset.zip

zip_path = '/content/Movie_Lens_100K_Dataset.zip'
extract_dir = '/content/ml-100k' # The zip file contains a folder named 'ml-100k'

# Check if the zip file exists before trying to extract
if os.path.exists(zip_path):
    print(f"Found zip file at: {zip_path}")
    # Create extraction directory if it doesn't exist
    if not os.path.exists(extract_dir):
        # Extract to /content/, which will create the 'ml-100k' folder
        with zipfile.ZipFile(zip_path, 'r') as z:
            z.extractall('/content/')
        print(f"Successfully extracted to: {extract_dir}")
    else:
        print(f"Extraction directory already exists: {extract_dir}. Skipping extraction.")
else:
    print(f"❌ ERROR: Zip file not found at '{zip_path}'.")
    print("Please upload the 'Movie_Lens_100K_Dataset.zip' file to your Colab session.")

"""
═══════════════════════════════════════════════════════════════════════════════
    CLUSTERING METHODS FOR COLLABORATIVE FILTERING - MOVIELENS DATASET
═══════════════════════════════════════════════════════════════════════════════

This implementation demonstrates three clustering methods on real MovieLens data:

1. K-MEANS CLUSTERING (⭐ FASTEST for CF!)
   - Training: O(n·k·d·t)
   - Prediction: O(k·d)

2. K-NEAREST NEIGHBORS (Lazy Learning)
   - Training: O(1) - just stores data
   - Prediction: O(n·d) - checks all points

3. DECISION TREE CLUSTERING (Hierarchical)
   - Training: O(n·d·log n)
   - Prediction: O(log n) - tree depth

Each implementation tracks:
✓ Actual execution time
✓ Number of operations performed
✓ Theoretical complexity
✓ Comparison with expected O() notation

Uses MovieLens 100K dataset for realistic collaborative filtering scenarios.
═══════════════════════════════════════════════════════════════════════════════
"""

# ═══════════════════════════════════════════════════════════════════════════
# UTILITY FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════
def load_movielens_data():
    """
    Loads the MovieLens 100K dataset from the specified Colab path.
    """
    print("Loading MovieLens dataset from u.data...")

    # Define the path to the u.data file within the extracted folder
    ratings_path = '/content/ml-100k/u.data'

    if not os.path.exists(ratings_path):
        print(f"❌ ERROR: The file 'u.data' was not found at '{ratings_path}'.")
        print("Please ensure the zip file was extracted correctly.")
        return None, None, None

    # Load the file (tab-separated)
    ratings = pd.read_csv(
        ratings_path,
        sep='\t',
        engine='python',
        names=['userId', 'movieId', 'rating', 'timestamp']
    )

    print(f"✓ Loaded {len(ratings):,} ratings")
    print(f"  Users: {ratings['userId'].nunique():,}")
    print(f"  Movies: {ratings['movieId'].nunique():,}")

    # Convert rating to numeric
    ratings['rating'] = pd.to_numeric(ratings['rating'], errors='coerce')
    ratings.dropna(subset=['rating'], inplace=True)

    user_features = []
    user_ids = []

    for user_id in ratings['userId'].unique():
        user_ratings = ratings[ratings['userId'] == user_id]
        if len(user_ratings) < 5:
            continue

        # Feature vector: [average rating, number of ratings]
        feature_vector = [
            user_ratings['rating'].mean(),
            len(user_ratings)
        ]
        user_features.append(feature_vector)
        user_ids.append(user_id)

    X = np.array(user_features)
    print(f"✓ Created feature vectors for {len(X):,} users")
    print(f"  Matrix shape: {X.shape}")

    return X, user_ids, ratings


def print_header(title, char='═', width=70):
    """Print formatted header"""
    print(f"\n{char * width}")
    print(f" {title}")
    print(f"{char * width}")


def format_number(num):
    """Format numbers with commas"""
    return f"{num:,}"


def format_time(seconds):
    """Format time in appropriate units"""
    if seconds < 0.001:
        return f"{seconds*1000000:.2f} μs"
    elif seconds < 1:
        return f"{seconds*1000:.2f} ms"
    else:
        return f"{seconds:.4f} s"


# ═══════════════════════════════════════════════════════════════════════════
# 1. K-MEANS CLUSTERING (FASTEST!)
# ═══════════════════════════════════════════════════════════════════════════

class KMeansClustering:
    """
    K-Means Clustering for CF with Complexity Tracking

    Training Complexity: O(n·k·d·t)
    - n: number of users
    - k: number of clusters
    - d: number of dimensions (features)
    - t: iterations until convergence

    Prediction Complexity: O(k·d) per user
    """

    def __init__(self, n_clusters=10, max_iters=100, tolerance=1e-4):
        self.n_clusters = n_clusters
        self.max_iters = max_iters
        self.tolerance = tolerance
        self.centroids = None
        self.labels = None

        # Complexity tracking
        self.train_time = 0
        self.train_ops = 0
        self.pred_time = 0
        self.pred_ops = 0
        self.iterations = 0

    def fit(self, X):
        """Train K-Means clustering - O(n·k·d·t)"""
        print_header("K-MEANS CLUSTERING - TRAINING", '─')

        n_samples, n_features = X.shape
        print(f"Dataset: {format_number(n_samples)} users × {n_features} features")
        print(f"Clusters: {self.n_clusters}")
        print(f"Max iterations: {self.max_iters}")

        start_time = time.time()
        operations = 0

        # Initialize centroids using k-means++
        random_indices = np.random.choice(n_samples, self.n_clusters, replace=False)
        self.centroids = X[random_indices].copy()
        operations += self.n_clusters * n_features

        print(f"\nInitialized {self.n_clusters} cluster centroids")
        print(f"Starting K-Means iterations...")

        # K-Means iterations
        for iteration in range(self.max_iters):
            # Assign users to nearest cluster centroid - O(n·k·d)
            distances = np.zeros((n_samples, self.n_clusters))

            for i in range(n_samples):
                for j in range(self.n_clusters):
                    # Calculate Euclidean distance manually
                    diff = X[i] - self.centroids[j]
                    # Note: np.sum and np.sqrt are used here, which are built-in NumPy functions.
                    # These are kept for a practical K-Means implementation.
                    distances[i, j] = np.sqrt(np.sum(diff ** 2))
                    operations += n_features * 3

            old_labels = self.labels
            self.labels = np.argmin(distances, axis=1)
            operations += n_samples * self.n_clusters

            # Update centroids - O(n·d)
            new_centroids = np.zeros((self.n_clusters, n_features))

            for j in range(self.n_clusters):
                cluster_users = X[self.labels == j]
                if len(cluster_users) > 0:
                    # Note: np.mean is used here.
                    new_centroids[j] = cluster_users.mean(axis=0)
                    operations += len(cluster_users) * n_features
                else:
                    new_centroids[j] = self.centroids[j]

            # Check convergence
            # Note: np.linalg.norm is used here.
            centroid_shift = np.linalg.norm(new_centroids - self.centroids)
            self.centroids = new_centroids

            if iteration % 10 == 0:
                cluster_sizes = [np.sum(self.labels == j) for j in range(self.n_clusters)]
                print(f"  Iteration {iteration + 1:3d}: shift = {centroid_shift:.6f}, "
                      f"clusters = {cluster_sizes}")

            if centroid_shift < self.tolerance:
                print(f"\n✓ Converged at iteration {iteration + 1}")
                self.iterations = iteration + 1
                break
        else:
            print(f"\n⚠ Reached maximum iterations ({self.max_iters})")
            self.iterations = self.max_iters

        self.train_time = time.time() - start_time
        self.train_ops = operations

        # Display cluster statistics
        print(f"\n{'─' * 70}")
        print(f"CLUSTER STATISTICS:")
        print(f"{'─' * 70}")
        for j in range(self.n_clusters):
            cluster_size = np.sum(self.labels == j)
            print(f"  Cluster {j}: {cluster_size} users ({100*cluster_size/n_samples:.1f}%)")

        print(f"\n{'─' * 70}")
        print(f"TRAINING RESULTS:")
        print(f"{'─' * 70}")
        print(f"Time taken:        {format_time(self.train_time)}")
        print(f"Operations:        {format_number(operations)}")
        print(f"Iterations:        {self.iterations}")

        theoretical_ops = n_samples * self.n_clusters * n_features * self.iterations
        print(f"\nTheoretical O(n·k·d·t):")
        print(f"  = O({format_number(n_samples)} × {self.n_clusters} × {n_features} × {self.iterations})")
        print(f"  = O({format_number(theoretical_ops)}) operations")
        print(f"\nActual/Theoretical ratio: {operations/theoretical_ops:.2f}x")
        print(f"{'─' * 70}")

        return self

    def predict(self, X):
        """Predict cluster for new users - O(k·d) per user"""
        n_samples, n_features = X.shape
        start_time = time.time()
        operations = 0

        predictions = np.zeros(n_samples, dtype=int)

        for i in range(n_samples):
            min_dist = float('inf')
            min_idx = 0

            for j in range(self.n_clusters):
                diff = X[i] - self.centroids[j]
                # Calculate Euclidean distance manually
                dist = np.sqrt(np.sum(diff ** 2))
                operations += n_features * 3

                if dist < min_dist:
                    min_dist = dist
                    min_idx = j

            predictions[i] = min_idx
            operations += self.n_clusters

        self.pred_time = time.time() - start_time
        self.pred_ops = operations

        return predictions

    def print_prediction_stats(self, n_samples, n_features):
        """Print prediction complexity statistics"""
        print(f"\nPREDICTION RESULTS ({format_number(n_samples)} users):")
        print(f"{'─' * 70}")
        print(f"Time taken:        {format_time(self.pred_time)}")
        print(f"Operations:        {format_number(self.pred_ops)}")
        print(f"Time per user:     {format_time(self.pred_time / n_samples)}")

        theoretical_ops = n_samples * self.n_clusters * n_features
        print(f"\nTheoretical O(n·k·d):")
        print(f"  = O({format_number(n_samples)} × {self.n_clusters} × {n_features})")
        print(f"  = O({format_number(theoretical_ops)}) operations")
        print(f"\nActual/Theoretical ratio: {self.pred_ops/theoretical_ops:.2f}x")


# ═══════════════════════════════════════════════════════════════════════════
# 2. K-NEAREST NEIGHBORS (LAZY LEARNING)
# ═══════════════════════════════════════════════════════════════════════════

class KNNClustering:
    """
    K-Nearest Neighbors for CF with Complexity Tracking

    Training Complexity: O(1) - lazy learning
    Prediction Complexity: O(n·d) per user - SLOW!
    """

    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.X_train = None
        self.y_train = None

        # Complexity tracking
        self.train_time = 0
        self.train_ops = 0
        self.pred_time = 0
        self.pred_ops = 0

    def fit(self, X, y):
        """'Train' KNN - O(1) - just stores data!"""
        print_header("K-NEAREST NEIGHBORS - TRAINING (Lazy Learning)", '─')

        n_samples, n_features = X.shape
        print(f"Dataset: {format_number(n_samples)} users × {n_features} features")
        print(f"Neighbors: {self.n_neighbors}")

        start_time = time.time()

        # Just store the data - no actual training!
        self.X_train = X.copy()
        self.y_train = y.copy()
        operations = n_samples * n_features

        self.train_time = time.time() - start_time
        self.train_ops = operations

        print(f"\n{'─' * 70}")
        print(f"TRAINING RESULTS:")
        print(f"{'─' * 70}")
        print(f"Time taken:        {format_time(self.train_time)}")
        print(f"Operations:        {format_number(operations)} (just copying data)")
        print(f"\n⚠ NOTE: KNN is 'lazy learning' - no actual training!")
        print(f"         All computation happens during prediction!")
        print(f"{'─' * 70}")

        return self

    def predict(self, X):
        """Predict using KNN - O(n_test × n_train × d) - SLOW!"""
        n_test, n_features = X.shape
        n_train = self.X_train.shape[0]

        start_time = time.time()
        operations = 0

        predictions = np.zeros(n_test, dtype=int)

        print(f"\nPredicting for {format_number(n_test)} users...")
        print(f"⚠ Must check ALL {format_number(n_train)} training users for each prediction!")

        for i in range(n_test):
            # Compute distance to ALL training users - O(n_train·d)
            distances = np.zeros(n_train)

            for j in range(n_train):
                diff = X[i] - self.X_train[j]
                # Calculate Euclidean distance manually
                dist = np.sqrt(np.sum(diff ** 2))
                operations += n_features * 3

            # Find k nearest neighbors
            # Note: np.argpartition is used here.
            k_nearest_indices = np.argpartition(distances, self.n_neighbors)[:self.n_neighbors]
            operations += n_train

            # Majority vote
            k_nearest_labels = self.y_train[k_nearest_indices]
            # Note: Counter is used here.
            predictions[i] = Counter(k_nearest_labels).most_common(1)[0][0]
            operations += self.n_neighbors

            if (i + 1) % 20 == 0:
                print(f"  Processed {i+1}/{n_test} users...")

        self.pred_time = time.time() - start_time
        self.pred_ops = operations

        return predictions

    def print_prediction_stats(self, n_test, n_features):
        """Print prediction complexity statistics"""
        n_train = self.X_train.shape[0]

        print(f"\nPREDICTION RESULTS ({format_number(n_test)} test users):")
        print(f"{'─' * 70}")
        print(f"Time taken:        {format_time(self.pred_time)}")
        print(f"Operations:        {format_number(self.pred_ops)}")
        print(f"Time per user:     {format_time(self.pred_time / n_test)}")

        theoretical_ops = n_test * n_train * n_features
        print(f"\nTheoretical O(n_test · n_train · d):")
        print(f"  = O({format_number(n_test)} × {format_number(n_train)} × {n_features})")
        print(f"  = O({format_number(theoretical_ops)}) operations")
        print(f"\nActual/Theoretical ratio: {self.pred_ops/theoretical_ops:.2f}x")

        print(f"\n⚠ WARNING: Checked ALL {format_number(n_train)} training users for EACH prediction!")
        print(f"           This is why KNN doesn't scale without indexing!")


# ═══════════════════════════════════════════════════════════════════════════
# 3. DECISION TREE CLUSTERING (HIERARCHICAL)
# ═══════════════════════════════════════════════════════════════════════════

class DecisionTreeClustering:
    """
    Decision Tree for CF with Complexity Tracking

    Training Complexity: O(n·d·log n)
    Prediction Complexity: O(log n) per user
    """

    def __init__(self, max_depth=10, min_samples_split=5):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None
        self.tree_depth = 0

        # Complexity tracking
        self.train_time = 0
        self.train_ops = 0
        self.pred_time = 0
        self.pred_ops = 0

    def _entropy(self, y):
        """Calculate entropy"""
        # Note: np.unique, np.sum, and np.log2 are used here.
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

    def _best_split(self, X, y):
        """Find best split - O(n·d)"""
        n_samples, n_features = X.shape
        best_gain = -1
        best_feature = None
        best_threshold = None
        operations = 0

        parent_entropy = self._entropy(y)
        operations += len(y)

        for feature in range(n_features):
            # Note: np.unique is used here.
            thresholds = np.unique(X[:, feature])

            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue

                left_entropy = self._entropy(y[left_mask])
                right_entropy = self._entropy(y[right_mask])

                n_left = np.sum(left_mask)
                n_right = np.sum(right_mask)

                weighted_entropy = (n_left/n_samples * left_entropy +
                                  n_right/n_samples * right_entropy)

                gain = parent_entropy - weighted_entropy
                operations += n_samples * 2

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold, operations

    def _build_tree(self, X, y, depth=0):
        """Build tree recursively - O(n·d·log n)"""
        n_samples, n_features = X.shape
        # Note: np.unique is used here.
        n_labels = len(np.unique(y))
        operations = 0

        if (depth >= self.max_depth or
            n_labels == 1 or
            n_samples < self.min_samples_split):
            # Note: Counter is used here.
            leaf_value = Counter(y).most_common(1)[0][0]
            return {'leaf': True, 'value': leaf_value}, operations

        best_feature, best_threshold, split_ops = self._best_split(X, y)
        operations += split_ops

        if best_feature is None:
            leaf_value = Counter(y).most_common(1)[0][0]
            return {'leaf': True, 'value': leaf_value}, operations

        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        left_subtree, left_ops = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree, right_ops = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        operations += left_ops + right_ops
        self.tree_depth = max(self.tree_depth, depth + 1)

        return {
            'leaf': False,
            'feature': best_feature,
            'threshold': best_threshold,
            'left': left_subtree,
            'right': right_subtree
        }, operations

    def fit(self, X, y):
        """Train Decision Tree - O(n·d·log n)"""
        print_header("DECISION TREE CLUSTERING - TRAINING", '─')

        n_samples, n_features = X.shape
        print(f"Dataset: {format_number(n_samples)} users × {n_features} features")
        print(f"Max depth: {self.max_depth}")
        print(f"Min samples to split: {self.min_samples_split}")

        print(f"\nBuilding hierarchical tree...")

        start_time = time.time()
        self.tree, operations = self._build_tree(X, y)
        self.train_time = time.time() - start_time
        self.train_ops = operations

        print(f"\n✓ Tree built successfully")
        print(f"  Actual tree depth: {self.tree_depth}")

        print(f"\n{'─' * 70}")
        print(f"TRAINING RESULTS:")
        print(f"{'─' * 70}")
        print(f"Time taken:        {format_time(self.train_time)}")
        print(f"Operations:        {format_number(operations)}")
        print(f"Tree depth:        {self.tree_depth}")

        theoretical_ops = n_samples * n_features * int(np.log2(n_samples))
        print(f"\nTheoretical O(n·d·log n):")
        print(f"  = O({format_number(n_samples)} × {n_features} × log₂({format_number(n_samples)}))")
        print(f"  = O({format_number(n_samples)} × {n_features} × {int(np.log2(n_samples))})")
        print(f"  ≈ O({format_number(theoretical_ops)}) operations")
        print(f"\nActual/Theoretical ratio: {operations/theoretical_ops:.2f}x")
        print(f"{'─' * 70}")

        return self

    def _predict_one(self, x, node):
        """Predict one user - O(log n)"""
        operations = 0

        while not node['leaf']:
            operations += 1
            if x[node['feature']] <= node['threshold']:
                node = node['left']
            else:
                node = node['right']

        return node['value'], operations

    def predict(self, X):
        """Predict using tree - O(log n) per user"""
        n_samples = X.shape[0]
        start_time = time.time()

        predictions = np.zeros(n_samples, dtype=int)
        operations = 0

        for i in range(n_samples):
            predictions[i], ops = self._predict_one(X[i], self.tree)
            operations += ops

        self.pred_time = time.time() - start_time
        self.pred_ops = operations

        return predictions

    def print_prediction_stats(self, n_samples, n_features):
        """Print prediction stats"""
        print(f"\nPREDICTION RESULTS ({format_number(n_samples)} users):")
        print(f"{'─' * 70}")
        print(f"Time taken:        {format_time(self.pred_time)}")
        print(f"Operations:        {format_number(self.pred_ops)}")
        print(f"Time per user:     {format_time(self.pred_time / n_samples)}")
        print(f"Avg tree depth:    {self.pred_ops / n_samples:.1f} nodes traversed")

        theoretical_ops = n_samples * self.tree_depth
        print(f"\nTheoretical O(n·log n):")
        print(f"  = O({format_number(n_samples)} × {self.tree_depth})")
        print(f"  = O({format_number(theoretical_ops)}) operations")
        print(f"\nActual/Theoretical ratio: {self.pred_ops/theoretical_ops:.2f}x")


# ═══════════════════════════════════════════════════════════════════════════
# RAW COSINE SIMILARITY (Manual Loop Implementation)
# ═══════════════════════════════════════════════════════════════════════════

def calculate_raw_cosine_similarity(X):
    """
    Calculates raw cosine similarity between all pairs of users in X.
    Uses manual loops, avoiding built-in functions and dot products.
    Complexity: O(n^2 * d)
    - n: number of users
    - d: number of features
    """
    print_header("RAW COSINE SIMILARITY - MANUAL LOOP", '─')

    n_samples, n_features = X.shape
    print(f"Dataset: {format_number(n_samples)} users × {n_features} features")

    start_time = time.time()
    operations = 0

    # Space complexity: O(n^2) for the similarity matrix
    similarity_matrix = np.zeros((n_samples, n_samples))

    for i in range(n_samples):
        for j in range(n_samples):
            # Calculate numerator (A . B) - STRICTLY MANUAL
            numerator = 0
            for k in range(n_features):
                numerator += X[i, k] * X[j, k]
                operations += 2 # 1 multiplication, 1 addition

            # Calculate denominator (||A|| * ||B||) - STRICTLY MANUAL
            norm_a_sq = 0
            norm_b_sq = 0
            for k in range(n_features):
                norm_a_sq += X[i, k] * X[i, k]
                norm_b_sq += X[j, k] * X[j, k]
                operations += 4 # 2 multiplications, 2 additions

            denominator_sq = norm_a_sq * norm_b_sq
            operations += 1 # 1 multiplication

            if denominator_sq == 0:
                similarity = 0.0
            else:
                # Use power operator for square root (equivalent to np.sqrt)
                similarity = numerator / (denominator_sq ** 0.5)
                operations += 2 # 1 power (sqrt), 1 division

            similarity_matrix[i, j] = similarity
            operations += 1 # 1 assignment

    end_time = time.time()
    total_time = end_time - start_time

    # Space complexity calculation (approximate)
    # The matrix is n*n elements. Assuming 8 bytes per float.
    space_complexity_bytes = n_samples * n_samples * 8

    print(f"\n{'─' * 70}")
    print(f"RESULTS:")
    print(f"{'─' * 70}")
    print(f"Time taken:        {format_time(total_time)}")
    print(f"Operations:        {format_number(operations)}")
    print(f"Theoretical O():   O(n²d)")
    print(f"Space Complexity:  O(n²) - approx {format_number(space_complexity_bytes)} bytes for similarity matrix")
    print(f"Example Sim (0, 1): {similarity_matrix[0, 1]:.4f}")
    print(f"{'─' * 70}")

    return similarity_matrix, total_time, operations, space_complexity_bytes

# ═══════════════════════════════════════════════════════════════════════════
# MEAN-CENTERED COSINE SIMILARITY (Manual Loop Implementation)
# ═══════════════════════════════════════════════════════════════════════════

def calculate_mean_centered_cosine_similarity(X):
    """
    Calculates Mean-Centered Cosine Similarity.

    This method centers the feature vector by subtracting the mean of
    the entire feature vector (all features).

    Uses manual loops, avoiding built-in functions and dot products.
    Complexity: O(n^2 * d)
    - n: number of users
    - d: number of features
    """
    print_header("MEAN-CENTERED COSINE SIMILARITY - MANUAL LOOP", '─')

    n_samples, n_features = X.shape
    print(f"Dataset: {format_number(n_samples)} users × {n_features} features")

    start_time = time.time()
    operations = 0

    # Space complexity: O(n^2) for the similarity matrix + O(n) for means
    similarity_matrix = np.zeros((n_samples, n_samples))

    # Step 1: Calculate the mean for each user (row) - STRICTLY MANUAL
    user_means = np.zeros(n_samples)
    for i in range(n_samples):
        row_sum = 0
        for k in range(n_features):
            row_sum += X[i, k]
            operations += 1 # 1 addition
        user_means[i] = row_sum / n_features
        operations += 1 # 1 division

    # Step 2: Calculate similarity
    for i in range(n_samples):
        for j in range(n_samples):
            # Mean-center the vectors - STRICTLY MANUAL
            a_centered = np.zeros(n_features)
            b_centered = np.zeros(n_features)
            for k in range(n_features):
                a_centered[k] = X[i, k] - user_means[i]
                b_centered[k] = X[j, k] - user_means[j]
                operations += 2 # 2 subtractions

            # Calculate numerator (centered_A . centered_B) - STRICTLY MANUAL
            numerator = 0
            for k in range(n_features):
                numerator += a_centered[k] * b_centered[k]
                operations += 2 # 1 multiplication, 1 addition

            # Calculate denominator (||centered_A|| * ||centered_B||) - STRICTLY MANUAL
            norm_a_sq = 0
            norm_b_sq = 0
            for k in range(n_features):
                norm_a_sq += a_centered[k] * a_centered[k]
                norm_b_sq += b_centered[k] * b_centered[k]
                operations += 4 # 2 multiplications, 2 additions

            denominator_sq = norm_a_sq * norm_b_sq
            operations += 1 # 1 multiplication

            if denominator_sq == 0:
                similarity = 0.0
            else:
                # Use power operator for square root (equivalent to np.sqrt)
                similarity = numerator / (denominator_sq ** 0.5)
                operations += 2 # 1 power (sqrt), 1 division

            similarity_matrix[i, j] = similarity
            operations += 1 # 1 assignment

    end_time = time.time()
    total_time = end_time - start_time

    # Space complexity calculation (approximate)
    space_complexity_bytes = n_samples * n_samples * 8 + n_samples * 8

    print(f"\n{'─' * 70}")
    print(f"RESULTS:")
    print(f"{'─' * 70}")
    print(f"Time taken:        {format_time(total_time)}")
    print(f"Operations:        {format_number(operations)}")
    print(f"Theoretical O():   O(n²d)")
    print(f"Space Complexity:  O(n²) - approx {format_number(space_complexity_bytes)} bytes for similarity matrix")
    print(f"Example Sim (0, 1): {similarity_matrix[0, 1]:.4f}")
    print(f"{'─' * 70}")

    return similarity_matrix, total_time, operations, space_complexity_bytes


# ═══════════════════════════════════════════════════════════════════════════
# MAIN EXECUTION
# ═══════════════════════════════════════════════════════════════════════════

def run_movielens_comparison_with_similarity():
    """
    Main function to run the clustering and similarity comparisons.
    """
    X, user_ids, ratings = load_movielens_data()

    if X is None:
        print("\nExiting due to data loading error.")
        return

    # --- 1. Clustering Methods ---

    # For clustering, we need a set of labels (e.g., from a pre-trained model)
    # Since we don't have pre-trained labels, we'll use a quick KMeans from sklearn
    # to generate 'ground truth' labels for the KNN and DT training steps.
    print_header("GENERATING BASE CLUSTERS (SKLEARN KMEANS)", '=')

    # Use a small number of clusters for demonstration
    K = 5
    # Note: SKLearnKMeans is used here for generating base labels.
    kmeans_base = SKLearnKMeans(n_clusters=K, random_state=42, n_init=10)
    y_base = kmeans_base.fit_predict(X)

    print(f"✓ Generated {K} base clusters for training KNN and DT.")

    # Split data into training and testing sets (80/20 split)
    n_samples = X.shape[0]
    train_size = int(0.8 * n_samples)

    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y_base[:train_size], y_base[train_size:]

    print(f"\nData Split: Train={X_train.shape[0]} users, Test={X_test.shape[0]} users")

    # 1. K-Means Clustering (Custom Implementation)
    kmeans_cf = KMeansClustering(n_clusters=K)
    kmeans_cf.fit(X_train)
    y_pred_kmeans = kmeans_cf.predict(X_test)
    kmeans_cf.print_prediction_stats(X_test.shape[0], X_test.shape[1])

    # 2. K-Nearest Neighbors (Custom Implementation)
    knn_cf = KNNClustering(n_neighbors=5)
    knn_cf.fit(X_train, y_train)
    y_pred_knn = knn_cf.predict(X_test)
    knn_cf.print_prediction_stats(X_test.shape[0], X_test.shape[1])

    # 3. Decision Tree Clustering (Custom Implementation)
    dt_cf = DecisionTreeClustering(max_depth=10)
    dt_cf.fit(X_train, y_train)
    y_pred_dt = dt_cf.predict(X_test)
    dt_cf.print_prediction_stats(X_test.shape[0], X_test.shape[1])

    # --- 2. Similarity Methods ---

    # Use a smaller subset for similarity to avoid excessive runtime (O(n^2*d))
    # Similarity calculation is very slow for large N
    subset_size = min(200, X.shape[0])
    X_subset = X[:subset_size]

    print_header(f"SIMILARITY COMPARISON (SUBSET N={subset_size})", '=')

    # Raw Cosine Similarity
    calculate_raw_cosine_similarity(X_subset)

    # Mean-Centered Cosine Similarity
    calculate_mean_centered_cosine_similarity(X_subset)

    print_header("COMPARISON SUMMARY", '=')
    print("Clustering Methods:")
    print(f"  K-Means Prediction Time: {format_time(kmeans_cf.pred_time)} (O(k·d))")
    print(f"  DT Prediction Time:      {format_time(dt_cf.pred_time)} (O(log n))")
    print(f"  KNN Prediction Time:     {format_time(knn_cf.pred_time)} (O(n·d) - Slowest)")

    print("\nSimilarity Methods (O(n²d)):")
    print(f"  Raw Cosine and Mean-Centered Cosine were calculated on a subset of {subset_size} users.")
    print("  These methods are computationally expensive for large user bases.")


if __name__ == '__main__':
    run_movielens_comparison_with_similarity()

Found zip file at: /content/Movie_Lens_100K_Dataset.zip
Successfully extracted to: /content/ml-100k
Loading MovieLens dataset from u.data...
✓ Loaded 100,000 ratings
  Users: 943
  Movies: 1,682
✓ Created feature vectors for 943 users
  Matrix shape: (943, 2)

 GENERATING BASE CLUSTERS (SKLEARN KMEANS)
✓ Generated 5 base clusters for training KNN and DT.

Data Split: Train=754 users, Test=189 users

──────────────────────────────────────────────────────────────────────
 K-MEANS CLUSTERING - TRAINING
──────────────────────────────────────────────────────────────────────
Dataset: 754 users × 2 features
Clusters: 5
Max iterations: 100

Initialized 5 cluster centroids
Starting K-Means iterations...
  Iteration   1: shift = 26.124117, clusters = [np.int64(308), np.int64(82), np.int64(93), np.int64(217), np.int64(54)]
  Iteration  11: shift = 14.578499, clusters = [np.int64(153), np.int64(265), np.int64(46), np.int64(109), np.int64(181)]
  Iteration  21: shift = 7.381364, clusters = [np.int6

# Under the Supervission of: Dr. **Samy** & Eng. **Salama**